In [4]:
!pip install keras

     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     --------------- ---------------------- 20.5/48.7 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 48.7/48.7 kB 486.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.1 MB 3.9 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.1 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/133.7 kB ? eta -:--:--
   ---------------------------------------- 133.7/133.7 kB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/127.5 kB ? eta -:--:--
   ---------------------------------------- 127.5/127.5 kB 7.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/267.2 kB ? eta -:--:--
   -------------------

In [8]:
!pip install tensorflow

   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/385.2 MB 2.2 MB/s eta 0:02:57
   ---------------------------------------- 0.5/385.2 MB 5.4 MB/s eta 0:01:11
   ---------------------------------------- 1.1/385.2 MB 7.9 MB/s eta 0:00:49
   ---------------------------------------- 1.8/385.2 MB 10.5 MB/s eta 0:00:37
   ---------------------------------------- 2.8/385.2 MB 12.6 MB/s eta 0:00:31
   ---------------------------------------- 3.4/385.2 MB 13.5 MB/s eta 0:00:29
   ---------------------------------------- 3.6/385.2 MB 11.5 MB/s eta 0:00:34
   ---------------------------------------- 4.8/385.2 MB 13.3 MB/s eta 0:00:29
    --------------------------------------- 5.7/385.2 MB 14.1 MB/s eta 0:00:27
    --------------------------------------- 6.7/385.2 MB 14.7 MB/s eta 0:00:26
    --------------------------------------- 7.5/385.2 MB 15.1 MB/s eta 0:00:26
    --------------------------------------- 8.4/385.2 MB 15.3 M

In [11]:
import numpy as np
from keras.datasets import mnist
import random

In [12]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [20]:
train_X=train_X.reshape(60000,784,1)    # flattening
test_X=test_X.reshape(10000,784,1)

train_y=train_y.reshape(60000,1)
test_y=test_y.reshape(10000,1)

train_X= train_X/255
test_X = test_X/255

train_X=train_X[:10000]         #taking the first 10000 images.
train_y=train_y[:10000]
test_X=test_X[:1000]
test_y=test_y[:1000]
train_data=list(zip(train_X,train_y))
test_data=list(zip(test_X,test_y))

ValueError: cannot reshape array of size 7840000 into shape (60000,784,1)

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [23]:
import numpy as np
import random

class Network(object):
    def __init__(self, sizes):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def forwardpropagation(self, a):
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a) + b)
        return a

    def backpropagation(self, x, y):
        y_t = np.zeros((10, 1))
        y_t[y] = 1

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        activation = x
        activation_list = [x]

        for w, b in zip(self.weights, self.biases):
            activation = self.sigmoid(np.dot(w, activation) + b)
            activation_list.append(activation)

        delta = activation_list[-1] - y_t
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activation_list[-2].T)

        for j in range(2, self.num_layers):
            sig_der = activation_list[-j] * (1 - activation_list[-j])
            delta = np.dot(self.weights[-j + 1].T, delta) * sig_der
            nabla_b[-j] = delta
            nabla_w[-j] = np.dot(delta, activation_list[-j - 1].T)

        return nabla_b, nabla_w

    def SGD(self, train_data, epochs, mini_batch_size, lr):
        n_train = len(train_data)
        for i in range(epochs):
            random.shuffle(train_data)
            mini_batches = [train_data[k:k + mini_batch_size] for k in range(0, n_train, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, lr)

            self.predict(train_data)
            print("Epoch {0} completed.".format(i + 1))

    def update_mini_batch(self, mini_batch, lr):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_b, delta_w = self.backpropagation(x, y)
            nabla_b = [nb + db for nb, db in zip(nabla_b, delta_b)]
            nabla_w = [nw + dw for nw, dw in zip(nabla_w, delta_w)]

        self.weights = [w - lr * nw / len(mini_batch) for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - lr * nb / len(mini_batch) for b, nb in zip(self.biases, nabla_b)]

    def predict(self, test_data):
        test_results = [(np.argmax(self.forwardpropagation(x)), y) for x, y in test_data]
        num = sum(int(x == y) for x, y in test_results)
        print("{0}/{1} classified correctly.".format(num, len(test_data)))

    def show(self):
      print(self.num_layers)
      for bias in self.biases:
          print(bias.shape)
      for weight in self.weights:
          print(weight.shape)

# Copy this show function from here. Paste it inside that Network Class.
# Comment out the show function here. Run this cell.

    def show(self):
      print(self.num_layers)
      for bias in self.biases:
          print(bias.shape)
      for weight in self.weights:
          print(weight.shape)

# Copy this show function from here. Paste it inside that Network Class.
# Comment out the show function here. Run this cell.

net=Network([784,128,64,10])
net.show()

4
(128, 1)
(64, 1)
(10, 1)
(128, 784)
(64, 128)
(10, 64)


In [21]:
net=Network([784,128,64,10])
print(train_X[0])
net.forwardpropagation(train_X[0])



[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.  

array([[1.03162935e-05],
       [3.20646464e-02],
       [6.22706619e-05],
       [3.53411373e-01],
       [9.79767874e-01],
       [6.12175179e-01],
       [2.27596831e-01],
       [3.49683920e-04],
       [2.11181090e-01],
       [9.98903427e-01]])

In [22]:
net=Network([784,128,64,10])
net.backpropagation(train_X[0],train_y[0])


([array([[ 3.40727737e-02],
         [-3.39309262e-03],
         [ 2.37081379e-03],
         [ 5.69638188e-01],
         [-1.79807447e-02],
         [ 3.33449942e-07],
         [ 1.87470957e-01],
         [-9.96975518e-02],
         [ 3.05246969e-02],
         [-9.35938249e-05],
         [-3.25124770e-01],
         [ 7.80502362e-11],
         [ 5.93186635e-03],
         [ 2.42570408e-01],
         [-3.04164819e-02],
         [ 5.33219081e-02],
         [ 7.39723301e-02],
         [-3.38334184e-01],
         [-8.10094876e-07],
         [ 3.11809181e-02],
         [ 6.18477658e-02],
         [ 6.96253601e-04],
         [-2.43662907e-05],
         [ 5.52739411e-06],
         [-1.70235989e-04],
         [ 1.13869246e-07],
         [-2.33032687e-04],
         [ 1.05083816e-07],
         [-5.50161281e-10],
         [-2.15762214e-03],
         [ 5.66689970e-02],
         [-1.77151463e-03],
         [ 1.05109190e-01],
         [-6.33128883e-07],
         [-8.35739049e-07],
         [-5.8579012

In [24]:
net=Network([784,128,64,10])
nabla_b,nabla_w=net.backpropagation(train_X[0],train_y[0])
for nb in nabla_b:
  print(nb.shape)
for nw in nabla_w:
  print(nw.shape)


(128, 1)
(64, 1)
(10, 1)
(128, 784)
(64, 128)
(10, 64)


In [25]:
net=Network([784,256,128,64,10])
net.SGD(train_data=train_data,epochs=20,mini_batch_size=20,lr=0.01)

C:\Users\Shobhan Sarkar\AppData\Local\Temp\ipykernel_19660\1124344037.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  num = sum(int(x == y) for x, y in test_results)


2407/10000 classified correctly.
Epoch 1 completed.
3585/10000 classified correctly.
Epoch 2 completed.
4397/10000 classified correctly.
Epoch 3 completed.
5038/10000 classified correctly.
Epoch 4 completed.
5520/10000 classified correctly.
Epoch 5 completed.
5872/10000 classified correctly.
Epoch 6 completed.
6145/10000 classified correctly.
Epoch 7 completed.
6403/10000 classified correctly.
Epoch 8 completed.
6641/10000 classified correctly.
Epoch 9 completed.
6825/10000 classified correctly.
Epoch 10 completed.
6970/10000 classified correctly.
Epoch 11 completed.
7107/10000 classified correctly.
Epoch 12 completed.
7226/10000 classified correctly.
Epoch 13 completed.
7330/10000 classified correctly.
Epoch 14 completed.
7416/10000 classified correctly.
Epoch 15 completed.
7501/10000 classified correctly.
Epoch 16 completed.
7582/10000 classified correctly.
Epoch 17 completed.
7644/10000 classified correctly.
Epoch 18 completed.
7727/10000 classified correctly.
Epoch 19 completed.
77

In [26]:
net=Network([784,128,64,10])
net.SGD(train_data=train_data,epochs=10,mini_batch_size=20,lr=0.01)
print("Test data:")
net.predict(test_data)

C:\Users\Shobhan Sarkar\AppData\Local\Temp\ipykernel_19660\1124344037.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  num = sum(int(x == y) for x, y in test_results)


2719/10000 classified correctly.
Epoch 1 completed.
3935/10000 classified correctly.
Epoch 2 completed.
4743/10000 classified correctly.
Epoch 3 completed.
5299/10000 classified correctly.
Epoch 4 completed.
5723/10000 classified correctly.
Epoch 5 completed.
6065/10000 classified correctly.
Epoch 6 completed.
6285/10000 classified correctly.
Epoch 7 completed.
6505/10000 classified correctly.
Epoch 8 completed.
6672/10000 classified correctly.
Epoch 9 completed.
6833/10000 classified correctly.
Epoch 10 completed.
Test data:
619/1000 classified correctly.
